In [3]:
#coding: utf-8


import numpy as np
import pandas as pd
import datetime
import math
from jqdata import *
import matplotlib.pyplot as plt


def filter_stock(codes,date):
    stock_list=[]
    for stock in codes:
        stock_info = get_security_info(stock)
        #print(stock)
        if stock_info.display_name != None:
            pause=get_price(stock,end_date=date, frequency='daily', fields='paused', fq='pre', count=2)['paused']
            #print(stock)
            #start_date=datetime.datetime.strptime(str(stock_info.start_date),'%Y-%m-%d')
            if 'ST' not in stock_info.name\
            and '*' not in stock_info.name\
            and '*' not in stock_info.display_name\
            and 'out' not in stock_info.display_name\
            and (date-stock_info.start_date) > datetime.timedelta(100)\
            and pause.values[0] != 1:
                stock_list.append(stock)
    return stock_list

In [39]:
trade_days=get_trade_days('2019-04-09','2019-05-09')
date=trade_days[-1]
stock_list = get_industry_stocks('801222') 
stock_list = filter_stock(stock_list,date) 
df=pd.DataFrame(index =stock_list, columns =['name','ROE','ROE2018','ROE2017','ROE2016',\
            'PB','PS',\
            'growth_rate','growth_rate2018','growth_rate2017','growth_rate2016',\
            'growth_profit_rate2018','growth_profit_rate2017','growth_profit_rate2016'])

for stk1 in stock_list:
    P = get_fundamentals(query(indicator.code, valuation.pb_ratio,valuation.ps_ratio,\
                               indicator.inc_total_revenue_year_on_year).filter(indicator.code==stk1))
    #RO = get_fundamentals_continuously(query(indicator.code, indicator.roe).filter(indicator.code == stk1))
    RO = get_fundamentals(query(indicator.code, indicator.roe).filter(indicator.code == stk1))
    RO_2018 = get_fundamentals(query(indicator.code, indicator.roe).filter(indicator.code == stk1),statDate=2018)
    RO_2017 = get_fundamentals(query(indicator.code, indicator.roe).filter(indicator.code == stk1),statDate=2017)
    RO_2016 = get_fundamentals(query(indicator.code, indicator.roe).filter(indicator.code == stk1),statDate=2016)
    GR_2018 = get_fundamentals(query(indicator.code, indicator.inc_total_revenue_year_on_year).filter(indicator.code == stk1),statDate=2018)
    GR_2017 = get_fundamentals(query(indicator.code, indicator.inc_total_revenue_year_on_year).filter(indicator.code == stk1),statDate=2017)
    GR_2016 = get_fundamentals(query(indicator.code, indicator.inc_total_revenue_year_on_year).filter(indicator.code == stk1),statDate=2016)
    GPR_2018 = get_fundamentals(query(indicator.code, indicator.inc_operation_profit_year_on_year).filter(indicator.code == stk1),statDate=2018)
    GPR_2017 = get_fundamentals(query(indicator.code, indicator.inc_operation_profit_year_on_year).filter(indicator.code == stk1),statDate=2017)
    GPR_2016 = get_fundamentals(query(indicator.code, indicator.inc_operation_profit_year_on_year).filter(indicator.code == stk1),statDate=2016)
    #print(RO_2018)
    df['name'][stk1]=get_security_info(stk1).display_name
    df['PB'][stk1]=P['pb_ratio'].values
    df['PS'][stk1]=P['ps_ratio'].values
    df['ROE'][stk1] = RO['roe'].values*4/100
    df['ROE2018'][stk1] = RO_2018['roe'].values/100
    df['ROE2017'][stk1] = RO_2017['roe'].values/100
    df['ROE2016'][stk1] = RO_2016['roe'].values/100
    df['growth_rate'][stk1] = P['inc_total_revenue_year_on_year'].values/100
    df['growth_rate2018'][stk1] = GR_2018['inc_total_revenue_year_on_year'].values/100
    df['growth_rate2017'][stk1] = GR_2017['inc_total_revenue_year_on_year'].values/100
    df['growth_rate2016'][stk1] = GR_2016['inc_total_revenue_year_on_year'].values/100
    df['growth_profit_rate2018'][stk1] = GPR_2018['inc_operation_profit_year_on_year'].values/100
    df['growth_profit_rate2017'][stk1] = GPR_2017['inc_operation_profit_year_on_year'].values/100
    df['growth_profit_rate2016'][stk1] = GPR_2016['inc_operation_profit_year_on_year'].values/100
    #print(df['PB'])
#ROE > -1 & growth_rate > -1 & pb > 0
df = df[(df['ROE'] > -1)&(df['growth_rate'] > -1)&(df['PS'] > 0)&(df['growth_profit_rate2018'] > -1)]

#print(df['growth_profit_rate2018'])

df["double_pb"] =  df.apply(lambda row: round(math.log(2.0 * row['PB']/(1+row['growth_rate']) , (1.0+row['ROE2018'])),2), axis=1)
df["double_ps"] =  df.apply(lambda row: round(math.log(2.0 * row['PS']/(1+row['growth_rate']) , (1.0+row['ROE2018'])),2), axis=1)
df["double_profit_pb"] =  df.apply(lambda row: round(math.log(2.0 * row['PB']/(1+row['growth_profit_rate2018']) , (1.0+row['ROE2018'])),2), axis=1)
df["double_profit_ps"] =  df.apply(lambda row: round(math.log(2.0 * row['PS']/(1+row['growth_profit_rate2018']) , (1.0+row['ROE2018'])),2), axis=1)
df = df[(df['double_pb'] < 50)&(df['double_pb'] > -50)]

df = df.sort("double_ps")
write_file('df.csv', df.to_csv(), append=False)
print(df['name'][50:])
#print(df)

300365.XSHE    恒华科技
600602.XSHG    云赛智联
300033.XSHE     同花顺
300609.XSHE    汇纳科技
300166.XSHE    东方国信
300448.XSHE    浩云科技
600728.XSHG    佳都科技
300496.XSHE    中科创达
300297.XSHE    蓝盾股份
002421.XSHE    达实智能
300682.XSHE    朗新科技
002063.XSHE    远光软件
300367.XSHE    东方网力
603232.XSHG    格尔软件
603990.XSHG    麦迪科技
300036.XSHE    超图软件
300451.XSHE    创业慧康
600588.XSHG    用友网络
300339.XSHE    润和软件
300659.XSHE    中孚信息
300248.XSHE     新开普
300075.XSHE    数字政通
002331.XSHE    皖通科技
300378.XSHE    鼎捷软件
300730.XSHE    科创信息
300150.XSHE    世纪瑞尔
603383.XSHG    顶点软件
300253.XSHE    卫宁健康
002530.XSHE    金财互联
600536.XSHG    中国软件
300399.XSHE     京天利
300168.XSHE    万达信息
300440.XSHE    运达科技
603508.XSHG    思维列控
002230.XSHE    科大讯飞
300380.XSHE    安硕信息
300379.XSHE     东方通
300288.XSHE    朗玛信息
300550.XSHE    和仁科技
300468.XSHE    四方精创
002380.XSHE    科远股份
300678.XSHE    中科信息
002153.XSHE    石基信息
600797.XSHG    浙大网新
000606.XSHE     顺利办
300663.XSHE    科蓝软件
300369.XSHE    绿盟科技
002405.XSHE    四维图新
300348.XSHE    长亮科技
002609.XSHE    捷顺科技


In [32]:
#coding: utf-8


import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from six import BytesIO
from jqdata import *
from scipy import signal  

from scipy.fftpack import fft,ifft
#from six import StringIO

In [33]:

def index_sum_cap(index,date):
    #a=get_index_weights(index, date)
    stock_list=get_index_stocks(index,date)
    #weight=np.array(a['weight'].values)
    #print(a)
    q=query(valuation.code, valuation.circulating_market_cap).filter(valuation.code.in_(stock_list))
    df = get_fundamentals(q, date)
    #print(len(weight),len(df['circulating_market_cap']),len(stock_list))
    sum_cap=df['circulating_market_cap'].sum()/len(stock_list)
    return sum_cap

data=pd.read_csv(BytesIO( read_file('df_index_hs300.csv')),index_col=[0])


In [45]:
start_time='2018-12-01'
end_time='2019-02-01'
trade_days=get_trade_days(start_time,end_time)
n=31
endtime=str(trade_days[n])+' 14:56:00'
#for i in range(1):
stock_list=get_index_stocks('000982.XSHG',date=trade_days[n])
p=get_price(stock_list,end_date=endtime, fields=['money','close'],frequency='1m', fq='pre',count=60)
p_limit=get_price(stock_list,end_date=trade_days[n], fields=['high_limit','low_limit'],frequency='1d', fq='pre',count=1)
a=p['close'].std(axis=0)
print(p['close'])
print(p_limit['low_limit'].values[0])

                     000006.XSHE  000008.XSHE  000009.XSHE  000012.XSHE  \
2019-01-17 13:57:00         5.31         3.99         4.46         4.16   
2019-01-17 13:58:00         5.31         3.99         4.47         4.16   
2019-01-17 13:59:00         5.31         3.99         4.47         4.17   
2019-01-17 14:00:00         5.31         4.00         4.47         4.17   
2019-01-17 14:01:00         5.29         4.00         4.47         4.17   
2019-01-17 14:02:00         5.31         4.00         4.47         4.16   
2019-01-17 14:03:00         5.31         4.00         4.47         4.16   
2019-01-17 14:04:00         5.30         4.00         4.45         4.16   
2019-01-17 14:05:00         5.31         3.99         4.47         4.17   
2019-01-17 14:06:00         5.32         3.99         4.47         4.17   
2019-01-17 14:07:00         5.31         3.99         4.46         4.16   
2019-01-17 14:08:00         5.28         3.99         4.46         4.17   
2019-01-17 14:09:00      

In [11]:
write_file('df_index_hs300.csv',data.to_csv(),append=False)